In [3]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Load the dataset from a CSV file
data = pd.read_csv("train.csv")

# Drop non-numeric columns (e.g., filename) before splitting
data = data.drop("filename", axis=1)
data = data.drop("length", axis=1)

# Define a dictionary to map genre labels to numerical values
genre_mapping = {
    'blues': 0,
    'classical': 1,
    'country': 2,
    'disco': 3,
    'hiphop': 4,
    'jazz': 5,
    'metal': 6,
    'pop': 7,
    'reggae': 8,
    'rock': 9
}

# Map genre labels to numerical values using the genre_mapping dictionary
data['label'] = data['label'].map(genre_mapping)

# Split the dataset into features and labels
X = data.drop("label", axis=1)  # Assuming "label" is the name of the target column
y = data["label"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape the input data for LSTM
X_train_lstm = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_lstm = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

# Create an RNN model with LSTM layers
rnn_model = Sequential()

# First LSTM layer with input shape
rnn_model.add(LSTM(128, activation='relu', return_sequences=True, input_shape=(1, X_train.shape[1])))
rnn_model.add(Dropout(0.2))  # Dropout layer to prevent overfitting

# Second LSTM layer
rnn_model.add(LSTM(64, activation='relu', return_sequences=True))
rnn_model.add(Dropout(0.2))  # Dropout layer to prevent overfitting

# Third LSTM layer
rnn_model.add(LSTM(32, activation='relu'))
rnn_model.add(Dropout(0.2))  # Dropout layer to prevent overfitting

# Dense layers
rnn_model.add(Dense(64, activation='relu'))
rnn_model.add(Dropout(0.2))  # Dropout layer to prevent overfitting

rnn_model.add(Dense(len(genre_mapping), activation='softmax'))

# Compile the model
rnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the RNN model
rnn_model.fit(X_train_lstm, y_train, epochs=50, batch_size=32, validation_split=0.1)

# Evaluate the RNN model
accuracy = rnn_model.evaluate(X_test_lstm, y_test, verbose=0)[1]
print(f"Accuracy: {accuracy:.2f}")

# Make predictions on the test data using RNN
y_pred_probs_rnn = rnn_model.predict(X_test_lstm)
y_pred_rnn = y_pred_probs_rnn.argmax(axis=1)

# Get a detailed classification report for RNN predictions
classification_rep_rnn = classification_report(y_test, y_pred_rnn)
print("Classification Report (RNN):\n", classification_rep_rnn)

# Calculate accuracy for RNN predictions
accuracy_rnn = accuracy_score(y_test, y_pred_rnn)
print(f"Accuracy (RNN): {accuracy_rnn:.2f}")

# Save the RNN model
rnn_model.save('rnn_model.h5')


Epoch 1/50
203/203 [==============================] - 7s 12ms/step - loss: 1.9005 - accuracy: 0.2994 - val_loss: 1.3762 - val_accuracy: 0.4972
Epoch 2/50
203/203 [==============================] - 2s 9ms/step - loss: 1.4114 - accuracy: 0.4890 - val_loss: 1.1279 - val_accuracy: 0.5889
Epoch 3/50
203/203 [==============================] - 2s 10ms/step - loss: 1.2353 - accuracy: 0.5532 - val_loss: 1.0341 - val_accuracy: 0.6167
Epoch 4/50
203/203 [==============================] - 2s 9ms/step - loss: 1.1138 - accuracy: 0.6072 - val_loss: 0.8897 - val_accuracy: 0.6736
Epoch 5/50
203/203 [==============================] - 2s 10ms/step - loss: 1.0116 - accuracy: 0.6412 - val_loss: 0.8215 - val_accuracy: 0.7167
Epoch 6/50
203/203 [==============================] - 2s 10ms/step - loss: 0.9519 - accuracy: 0.6721 - val_loss: 0.7763 - val_accuracy: 0.7486
Epoch 7/50
203/203 [==============================] - 2s 10ms/step - loss: 0.8736 - accuracy: 0.6899 - val_loss: 0.7450 - val_accuracy: 0.7444
E

In [5]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import load_model

# Define a dictionary to map genre labels to numerical values
genre_mapping = {
    'blues': 0,
    'classical': 1,
    'country': 2,
    'disco': 3,
    'hiphop': 4,
    'jazz': 5,
    'metal': 6,
    'pop': 7,
    'reggae': 8,
    'rock': 9
}

# Reverse the dictionary to perform inverse mapping
reverse_genre_mapping = {v: k for k, v in genre_mapping.items()}

# Load the test dataset from a CSV file
test_data = pd.read_csv("test.csv")
test_data_original_id = test_data['id']

# Drop non-numeric columns (e.g., id) before making predictions
test_data = test_data.drop("id", axis=1)
test_data = test_data.drop("length", axis=1)

# Scale the features using the same scaler used for training data
scaler = StandardScaler()
X_test_data_scaled = scaler.fit_transform(test_data)  # Assuming 'test_data' contains only features

# Reshape the input data for LSTM
X_test_data_scaled_lstm = X_test_data_scaled.reshape(X_test_data_scaled.shape[0], 1, X_test_data_scaled.shape[1])

# Create an RNN model with LSTM layers
rnn_model = Sequential()
rnn_model.add(LSTM(128, activation='relu', input_shape=(1, X_test_data_scaled.shape[1])))
rnn_model.add(Dense(10, activation='softmax'))  # Output layer with 10 units for 10 genres

# Compile the RNN model
rnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Load the trained RNN model
rnn_model = load_model('rnn_model.h5')

# Make predictions using the trained RNN model
y_pred_probabilities_rnn = rnn_model.predict(X_test_data_scaled_lstm)

# Get the predicted class labels for RNN
y_pred_test_rnn = np.argmax(y_pred_probabilities_rnn, axis=1)

# Create a new DataFrame with the original 'id' and predicted labels
predictions_df_rnn = pd.DataFrame({"id": test_data_original_id, "label": y_pred_test_rnn})

# Save the predictions to a CSV file
predictions_df_rnn.to_csv("predictions_rnn1.csv", index=False)

# Print a message indicating successful prediction and saving of the results for RNN
print("RNN Predictions have been made and saved to 'predictions_rnn.csv'.")


32/32 [==============================] - 0s 3ms/step
RNN Predictions have been made and saved to 'predictions_rnn.csv'.
